In [ ]:
import numpy as np
import struct
import cv2


In [26]:


class LidarPoint:
    def __init__(self, x, y, z, r):
        self.x = x
        self.y = y
        self.z = z
        self.r = r

def write_pod(out, t):
    out.write(struct.pack('dddd', t.x, t.y, t.z, t.r))

def read_pod(in_file):
    data = in_file.read(struct.calcsize('dddd'))
    if data:
        return struct.unpack('dddd', data)
    else:
        return None

def read_pod_vector(in_file):
    size = struct.unpack('l', in_file.read(struct.calcsize('l')))[0]
    vect = []
    for _ in range(size):
        t = read_pod(in_file)
        if t:
            vect.append(LidarPoint(*t))
    return vect

def write_pod_vector(out, vect):
    size = len(vect)
    out.write(struct.pack('l', size))
    for t in vect:
        write_pod(out, t)

def read_lidar_pts(file_name):
    with open(file_name, 'rb') as in_file:
        return read_pod_vector(in_file)


In [29]:
curr_lidar_pts = read_lidar_pts("first.dat")
prev_lidar_pts = read_lidar_pts("prev.dat")
# Assuming you already have the lidar points stored in the curr_lidar_pts list

for point in curr_lidar_pts:
    print(f"X: {point.x}, Y: {point.y}, Z: {point.z}, Reflectivity: {point.r}")



X: 8.015999794006348, Y: 0.01899999938905239, Z: -1.0299999713897705, Reflectivity: 0.36000001430511475
X: 8.015999794006348, Y: 0.03099999949336052, Z: -1.0299999713897705, Reflectivity: 0.3499999940395355
X: 8.02299976348877, Y: 0.0560000017285347, Z: -1.031999945640564, Reflectivity: 0.3700000047683716
X: 8.017000198364258, Y: 0.08100000023841858, Z: -1.031000018119812, Reflectivity: 0.33000001311302185
X: 8.02299976348877, Y: 0.10700000077486038, Z: -1.031999945640564, Reflectivity: 0.3799999952316284
X: 8.038000106811523, Y: 0.13199999928474426, Z: -1.034000039100647, Reflectivity: 0.3499999940395355
X: 8.02400016784668, Y: 0.15700000524520874, Z: -1.031999945640564, Reflectivity: 0.3400000035762787
X: 8.03499984741211, Y: 0.18299999833106995, Z: -1.034000039100647, Reflectivity: 0.3400000035762787
X: 8.038999557495117, Y: 0.19499999284744263, Z: -1.034000039100647, Reflectivity: 0.3799999952316284
X: 8.039999961853027, Y: 0.22100000083446503, Z: -1.034999966621399, Reflectivity: 

In [30]:
def compute_ttc_lidar(lidar_points_prev, lidar_points_curr):
    # Auxiliary variables
    dT = 0.1  # Time between two measurements in seconds
    lane_width = 4.0  # Assumed width of the ego lane

    # Find closest distance to Lidar points within ego lane
    min_x_prev = 1e9
    min_x_curr = 1e9

    for point in lidar_points_prev:
        # 3D points within the ego-lane
        if abs(point.y) <= (lane_width / 2.0):
            min_x_prev = min(min_x_prev, point.x)

    for point in lidar_points_curr:
        # 3D points within the ego-lane
        if abs(point.y) <= (lane_width / 2.0):
            min_x_curr = min(min_x_curr, point.x)

    # Compute TTC from both measurements
    if min_x_prev - min_x_curr == 0:
        return float('inf')  # To avoid division by zero
    else:
        return min_x_curr * dT / (min_x_prev - min_x_curr)

# Usage
TTC = compute_ttc_lidar(prev_lidar_pts, curr_lidar_pts)
print("TTC (Lidar):", TTC)


TTC (Lidar): 12.972158904366587
